
# Note book for the project 2 

Kaggle competition link: [Submition]('https://www.kaggle.com/c/epfml17-text/submit')

## Pipeline: 


### Create cooc matrix

1. sh build_vocab.sh
2. sh cut_vocab.sh
3. python3 pickle_vocab.py
4. python3 cooc.py

Now given the co-occurrence matrix and the vocabulary, it is not hard to train GloVe word embeddings, that is to compute an embedding vector for wach word in the vocabulary. We suggest to implement SGD updates to train the matrix factorization, as in

5. python3 glove_template.py

Once you tested your system on the small set of 10% of all tweets, we suggest you run on the full datasets pos_train_full.txt, neg_train_full.txt

### Building a Text Classifier:

1. Construct Features for the Training Texts: Load the training tweets and the built GloVe word embeddings. Using the word embeddings, construct a feature representation of each training tweet (by averaging the word vectors over all words of the tweet).

2. Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels indicate if a tweet used to contain a 🙂 or 🙁 smiley.

3. Prediction: Predict labels for all tweets in the test set.

4. Submission / Evaluation: Submit your predictions to kaggle, and verify the obtained misclassification error score. (You can also use a local separate validation set to get faster feedback on the accuracy of your system). Try to tune your system for best evaluation score.

### Extensions:
Naturally, there are many ways to improve your solution, both in terms of accuracy and computation speed. More advanced techniques can be found in the recent literature.



Importing usefull library

In [45]:
import numpy as np
import pandas as pd
%matplotlib inline
#!/usr/bin/env python3
from scipy.sparse import *
from sklearn import svm
import numpy as np
import pickle
import random

# 0.Variables


In [27]:
file_storing_ts = 'embeddings.npy'
file_storing_pos_ts_tweets = 'train_pos.txt'
file_storing_neg_ts_tweets = 'train_neg.txt'
file_storing_te_tweets = 'test_data.txt'
nb_dim = 20



In [24]:
def word_embeddings(file_name='cooc.pkl', destination='embeddings.npy'):
    print("loading cooccurrence matrix")
    with open(file_name, 'rb') as f:
        cooc = pickle.load(f)
    print("{} nonzero entries".format(cooc.nnz))
    
    nmax = 100
    print("using nmax =", nmax, ", cooc.max() =", cooc.max())
    print("initializing embeddings")
    embedding_dim = 20
    xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
    ys = np.random.normal(size=(cooc.shape[1], embedding_dim))
    eta = 0.001
    alpha = 3 / 4
    epochs = 10
    
    for epoch in range(epochs):
        print("epoch {}".format(epoch))
        for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
            
            f = ((n / nmax)**alpha) if n < nmax else 1
            inter_cost = (xs[ix]@(ys[jy]) - np.log(n))
            # We compute the gradients for both context and main vector words
            grad_main = f * inter_cost * ys[jy]
            grad_context = f * inter_cost * xs[ix]
    
            # Update the vector words
            xs[ix] = xs[ix] - (eta * grad_main)
            ys[jy] = ys[jy] - (eta * grad_context)
            
    np.save(destination, xs)
 


# 1.Construct words_embeddings for training set


In [ ]:
word_embeddings()


# 2.Load words for training set


In [16]:
with open('vocab.pkl', 'rb') as f :
    vocab = pickle.load(f)
xs = np.load(file_storing_ts)



# 3.Construct features for training tweets


In [40]:
pos_tweets = open(file_storing_pos_ts_tweets, 'r').readlines() 
neg_tweets = open(file_storing_neg_ts_tweets, 'r').readlines()


#Construct features for positive tweets
pos_features = []

for line in pos_tweets:
    sum_w = np.zeros(nb_dim)
    count = 0
    for word in line.split():
        local_w = vocab.get(word, -1)
        if local_w != -1:
            count += 1
            sum_w += xs[local_w]
     
    # If no match between the vocab and the tweet, we set the feature vector to zeros (zero best value ?)
    if(count == 0):
        count = 1
    pos_features.append(sum_w/count)
pos_features = np.array(pos_features)    
    
#Construct features for negative tweets
neg_features = []

for line in neg_tweets:
    sum_w = np.zeros(nb_dim)
    count = 0
    for word in line.split():
        local_w = vocab.get(word, -1)
        if local_w != -1:
            count += 1
            sum_w += xs[local_w]
     
    # If no match between the vocab and the tweet, we set the feature vector to zeros (zero best value ?)
    if(count == 0):
        count = 1
    neg_features.append(sum_w/count)
    
neg_features = np.array(neg_features)


# Train the linear classifier


In [50]:
clf = svm.SVC(max_iter=1000);
X = np.concatenate((pos_features, neg_features))
Y = np.concatenate((np.ones(len(pos_features)), np.full(len(neg_features), -1)))
clf.fit(X, Y)
                   

C:\Users\Lucas\Anaconda3\lib\site-packages\sklearn\svm\base.py:220: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=1000, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)